In [1]:
import pandas as pd
import requests

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from ast import literal_eval

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate


import warnings; warnings.simplefilter('ignore')

In [2]:
# importing inventory dataset

In [3]:
import joblib
inventory=joblib.load('inventorydata.pkl')

In [4]:
inventory

,id,code,inventory_id,inventory_name,name,search_name,display_name,description,image1,base_uom_name,category_name,sub_category_name,group_name,material_name,static_group_name,brand_name,division_name,sales_uom_name
0,26245,VRNT43101,BSNU1000,None,EASTERN GINGER PICKLE 400GM,EASTERN GINGER PICKLE 400GM,EASTERN GINGER PICKLE 400GM,EASTERN GINGER PICKLE 400GM,https://rgc-marketplace.s3.ap-south-1.amazonaw...,PCS,CANNED FOOD,CANNED FOOD,PICKLES,PICKLES,None,EASTERN,FOOD CUPBOARD,PCS
1,8531,VRNT20682,BSNU1000,None,GARNIER COLOR NATURALS 1 BLACK,,GARNIER COLOR NATURALS 1 BLACK,,https://rgc-marketplace.s3.ap-south-1.amazonaw...,PCS,HEALTH AND BEAUTY,HAIR CARE,HAIR COLOUR,None,None,MAGIC,NON FOOD CUPBOARD,PCS
2,33393,VRNT47250,BSNU1000,None,GAULOISES BLONDES LIGHT 20S,GAULOISES BLONDES LIGHT 20S,GAULOISES BLONDES LIGHT 20S,GAULOISES BLONDES LIGHT 20S,https://rgc-marketplace.s3.ap-south-1.amazonaw...,BOX,CIGARETTES AND SHISHA,CIGARETTES,LIGHT AND MENTHOL CIGARETTES,LIGHT AND MENTHOL CIGARETTES,None,GAULOISES,NON FOOD CUPBOARD,BOX
3,43094,VRNT55062,BSNU1000,None,CAVALLO SYNC CIGARETTE 20S,CAVALLO SYNC CIGARETTE 20S,CAVALLO SYNC CIGARETTE 20S,CAVALLO SYNC CIGARETTE 20S,https://rgc-marketplace.s3.ap-south-1.amazonaw...,PCS,CIGARETTES AND SHISHA,CIGARETTES,NORMAL CIGARETTES,NORMAL CIGARETTES,None,CAVALLO,NON FOOD CUPBOARD,OUTER
4,8322,VRNT20473,BSNU1000,None,TASTY FOOD CHANADAL TF 500GM,,TASTY FOOD CHANADAL TF 500GM,,https://rgc-marketplace.s3.ap-south-1.amazonaw...,PCS,HOME BAKING AND COOKING,HOME BAKING AND COOKING,PULSES,None,None,TASTY FOOD,FOOD CUPBOARD,PCS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31620,20733,VRNT35353,BSNU1000,None,MR/MUSCL TOILT/DUK/ORIGNL500ML,MR/MUSCL TOILT/DUK/ORIGNL500ML,MR/MUSCL TOILT/DUK/ORIGNL500ML,MR/MUSCL TOILT/DUK/ORIGNL500ML,https://rgc-marketplace.s3.ap-south-1.amazonaw...,PCS,HOME CLEANING LIQUIDS,HOME CLEANING LIQUIDS,TOILET CLEANERS,TOILET CLEANERS,None,MR.MUSCLE,NON FOOD CUPBOARD,PCS
31621,33846,VRNT44545,BSNU1008,None,AL ISLAMI CHICKEN SHEESH KEBAB 280GX12PCS,AL ISLAMI CHICKEN SHEESH KEBAB 280GX12PCS,AL ISLAMI CHICKEN SHEESH KEBAB 280GX12PCS,AL ISLAMI CHICKEN SHEESH KEBAB 280GX12PCS,https://rgc-marketplace.s3.ap-south-1.amazonaw...,CTN,FROZEN FOOD,FROZEN FOOD,FROZEN READY MEALS OTHERS,FROZEN READY MEALS OTHERS,None,AL ISLAMI,FOOD CUPBOARD,CTN
31622,18603,VRNT33201,BSNU1000,None,SADIA CHICKEN LIVER 3X450GM,SADIA CHICKEN LIVER 3X450GM,SADIA CHICKEN LIVER 3X450GM,SADIA CHICKEN LIVER 3X450GM,https://rgc-marketplace.s3.ap-south-1.amazonaw...,PCS,FROZEN FOOD,FROZEN FOOD,FROZEN MULUKHIYAH,FROZEN MULUKHIYAH,None,SADIA,FOOD CUPBOARD,PCS
31623,19016,VRNT33614,BSNU1000,None,ALALALI CORN FLOUR 450GM,ALALALI CORN FLOUR 450GM,ALALALI CORN FLOUR 450GM,ALALALI CORN FLOUR 450GM,https://rgc-marketplace.s3.ap-south-1.amazonaw...,PCS,HOME BAKING AND COOKING,BAKING INGREDIENTS,BAKING FLOUR AND MIXES,BAKING FLOUR AND MIXES,None,AL ALALI,FOOD CUPBOARD,PCS


In [5]:
# importing purchase History dataset from url

In [6]:
# all_data = []
# def get_paginated_results(url,all_data):
#     response = requests.get(url)
#     #print(response)
#     res = response.json()
#     if response.status_code == 200:
#         if res['data']["results"]:
#             all_data += res['data']["results"]
#     if res['data'] == None or res['data']['next'] == None:
#         return all_data
    
#     return get_paginated_results(res['data']['next'],all_data)

In [7]:
# url='https://api-designlayoutproduction-uat.hilalcart.com/dynamic/purchase-history-list'
# all_data_res = get_paginated_results(url,all_data)
# # print(all_data_res)


In [8]:
# purchaseData = pd.DataFrame(all_data_res)
# purchaseData

NameError: name 'all_data_res' is not defined

In [ ]:
#store variant id in id column

In [ ]:
purchaseData.id=purchaseData.variant_id
purchaseData

In [ ]:
purchaseData.columns

In [ ]:
#creating an purchase id with required column seperately

In [ ]:
purchaseId=purchaseData[["id","customer_code","created_year",'created_month','created_day']]
purchaseId

In [ ]:
#converting id column into int to merge

In [ ]:
purchaseId.id=purchaseId.id.astype('int')

In [ ]:
inventory.id=inventory.id.astype("int")

In [ ]:
#merging id with inventory to obtain datas of purchase id

In [ ]:
df=purchaseId.merge(inventory, on = 'id', how = 'left')

In [ ]:
df

In [9]:
df=joblib.load("product_list.pkl")

In [ ]:
df

In [ ]:
df.customer_code.unique()

In [ ]:
#function to take data from specific customer

In [10]:
def customer(code):
    d=df[df["customer_code"]==code]
    d=d.sort_values(by=['created_year',"created_month","created_day"],ascending=False)
    d.drop_duplicates(subset=["id"],inplace=True)
    l=list(d["id"])
    limit=l[0:10]
    return limit

In [ ]:
d=df[df["customer_code"]=="RGC1664003938"]
d=d.sort_values(by=['created_year',"created_month","created_day"],ascending=False)
d.drop_duplicates(subset=["id"],inplace=True)
l=list(d["id"])
limit=l[0:10]

In [ ]:
d

In [ ]:
d.id.nunique()

In [ ]:
#give customer code to get productid of recent purchases

In [11]:
customer("RGC1658833657")

[31927, 31966, 4492, 16752, 23867, 43452, 3142]

In [ ]:
#creating pickle file for dataset

In [ ]:
import joblib
joblib.dump(df,"product_list.pkl")

In [ ]:
#just to see customers

In [ ]:
df["customer_code"].unique()